# Cotas para Exercício da Atividade Parlamentar dos Senadores (CEAPS)

Segundo o site da Câmara dos Deputados (https://www2.camara.leg.br/transparencia/acesso-a-informacao/copy_of_perguntas-frequentes/cota-para-o-exercicio-da-atividade-parlamentar), a Cota para o Exercício da Atividade Parlamentar – CEAP (antiga verba indenizatória) é uma cota única mensal destinada a custear os gastos dos deputados exclusivamente vinculados ao exercício da atividade parlamentar.

O Ato da Mesa nº 43 de 2009, que detalha as regras para o uso da CEAP, determina que só podem ser indenizadas despesas com:
* passagens aéreas;
* telefonia; 
* serviços postais; 
* manutenção de escritórios de apoio à atividade parlamentar; 
* assinatura de publicações; 
* fornecimento de alimentação ao parlamentar; 
* hospedagem;
* outras despesas com locomoção, contemplando locação ou fretamento de aeronaves, veículos automotores e embarcações, serviços de táxi, pedágio e estacionamento e passagens terrestres, marítimas ou fluviais; 
* combustíveis e lubrificantes; 
* serviços de segurança; 
* contratação de consultorias e trabalhos técnicos; 
* divulgação da atividade parlamentar, exceto nos 120 dias anteriores às eleições; 
* participação do parlamentar em cursos, palestras, seminários, simpósios, congressos ou eventos congêneres; 
* e a complementação do auxílio-moradia.

O valor máximo mensal da cota depende da unidade da federação que o deputado representa. Essa variação ocorre por causa das passagens aéreas e está relacionada ao valor do trecho entre Brasília e o Estado que o deputado representa.

## Problemas de Pesquisa

1) Quem são os TOP 5 maiores gastos de cada ano?

        1.1) Podemos detalhar com o que foram realizados esses gastos?

2) Com qual tipo de despesa foi gasto mais dinheiro?

3) Qual será a melhor forma para analisar se um gasto pode ser categorizado como suspeito?

        3.1) Podemos criar um filtro para gastos suspeitos?

## Bibliotecas

In [1]:
import pandas as pd
import warnings
from IPython.display import display, Markdown, HTML
import locale
from babel.numbers import format_currency
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline
warnings.filterwarnings('ignore')

## Funções
Nesta sessão ficarão as funções criadas para este projeto.

In [2]:
# Função para concatenar dataframes
def concat(dataframes_list):
    df_concat = pd.concat(dataframes_list, ignore_index=True)
    return df_concat

# Função para selecionar o ano
def select_year(df, year):
    year_df = df[df['ANO'] == year]
    return year_df

# Função para criar DF contagem de solicitações por senador
def count(df, ano):
    contagem_ano = df.query("ANO == @ano")
    df_ano = contagem_ano.sort_values("CONTAGEM", ascending=False)
    return df_ano
    
# Função para criar DF Valor Total por senador
def value_sum(df):
    df_agrupado = df.groupby('SENADOR')['VALOR_REEMBOLSADO'].sum()
    df_agrupado = df_agrupado.to_frame()
    df_agrupado = df_agrupado.rename(columns={'SENADOR': 'SENADOR', 'VALOR_REEMBOLSADO': 'VALOR_REEMBOLSADO'})
    df_agrupado = df_agrupado.sort_values("VALOR_REEMBOLSADO", ascending=False)
    return df_agrupado

# Função para criar DF Valor Total por Tipo de Despesa
def value_type(df):
    df_agrupado = df.groupby('TIPO_DESPESA')['VALOR_REEMBOLSADO'].sum()
    df_agrupado = df_agrupado.to_frame()
    df_agrupado = df_agrupado.rename(columns={'TIPO_DESPESA': 'TIPO_DESPESA','VALOR_REEMBOLSADO': 'VALOR_TOTAL'})
    df_agrupado = df_agrupado.sort_values("VALOR_TOTAL", ascending=False)
    return df_agrupado

# Função para mostrar o valor total por ano
def total_value(df):
    locale.setlocale(locale.LC_ALL, 'pt_BR.UTF-8')
    soma_total = df['VALOR_TOTAL'].sum()
    valor_formatado = format_currency(soma_total, 'BRL', format='#,##0.00', locale='pt_BR')
    print("Soma total dos valores do ano de 2020:", valor_formatado)    

# Função para realizar a porcentagem de nulos por coluna
def nulls(df):
    nulls_percent = df.isnull().mean() * 100
    return nulls_percent

# Função para tratar valores da coluna VALOR_REEMBOLSADO e substituir a vírgula por ponto
def treat_value(df):
    df.loc[:, 'VALOR_REEMBOLSADO'] = df.loc[:, 'VALOR_REEMBOLSADO'].str.replace(',', '.').astype(float)
    
# Função para centralizar o dataframe
def center_df(df):
    df = df.style.set_table_attributes('style="margin-left: auto; margin-right: auto;"')
    return HTML(df.render())
    
# Função para inserir título
def title(title, size):
    return Markdown(f'<h1 style="font-size:{size}px; text-align:center;">{title}</h1>')

# FUnção para alterar tipo coluna para data
def todate(df):
    df_concat['DATA'] = pd.to_datetime(df_concat['DATA'], format='%d/%m/%Y', errors='coerce')
    return df['DATA']

# Função remover colunas
def drop(df, columns_list):
    df = df.drop(columns=columns_list)
    return df

## Importação de Dados

Os dados que serão utilizados são referentes aos anos de 2018 a 2022, disponíveis no site:
https://www12.senado.leg.br/transparencia/dados-abertos-transparencia/dados-abertos-ceaps

In [3]:
# Importando dados
data_2018 = pd.read_csv('data/despesa_ceaps_2018.csv', encoding='latin-1', sep=';', skiprows=[0])
data_2019 = pd.read_csv('data/despesa_ceaps_2019.csv', encoding='latin-1', sep=';', skiprows=[0])
data_2020 = pd.read_csv('data/despesa_ceaps_2020.csv', encoding='latin-1', sep=';', skiprows=[0])
data_2021 = pd.read_csv('data/despesa_ceaps_2021.csv', encoding='latin-1', sep=';', skiprows=[0])
data_2022 = pd.read_csv('data/despesa_ceaps_2022.csv', encoding='latin-1', sep=';', skiprows=[0])

## Concatenação das Tabelas 2018 - 2022
Para análise exploratória e comparação dos dados iremos agrupá-los em um único arquivo. 
Em alguns momentos ainda os dados separados por anos ainda serão utilizados.

In [4]:
df_concat = concat([data_2018, data_2019, data_2020, data_2021, data_2022] )
""" Comentários:
    Corrigir os valores da coluna VALOR_REEMBOLSADO (trocar , por .)
    Todos os dados que não for possível de realizar sua categorização serão excluídos desta análise (DOCUMENTO, COD_DOCUMENTO)"""
df_concat

,ANO,MES,SENADOR,TIPO_DESPESA,CNPJ_CPF,FORNECEDOR,DOCUMENTO,DATA,DETALHAMENTO,VALOR_REEMBOLSADO,COD_DOCUMENTO
0,2018,1,ACIR GURGACZ,"Aluguel de imóveis para escritório político, c...",004.948.028-63,GILBERTO PISELO DO NASCIMENTO,NaN,02/01/2018,Despesa com pagamento de aluguel de imóvel par...,6000,2088462
1,2018,1,ACIR GURGACZ,"Passagens aéreas, aquáticas e terrestres nacio...",63.764.211/0001-10,TRANSCONTINENTAL AGÊNCIA DE VIAGENS LTDA,M2CJMM,02/01/2018,"Companhia Aérea: AVIANCA, Localizador: M2CJMM....","274,99",2088476
2,2018,1,ACIR GURGACZ,"Passagens aéreas, aquáticas e terrestres nacio...",63.764.211/0001-10,TRANSCONTINENTAL AGÊNCIA DE VIAGENS LTDA,SFEXYF,02/01/2018,"Companhia Aérea: LATAM, Localizador: SFEXYF. P...","1094,29",2088528
3,2018,1,ACIR GURGACZ,"Passagens aéreas, aquáticas e terrestres nacio...",63.764.211/0001-10,TRANSCONTINENTAL AGÊNCIA DE VIAGENS LTDA,V64HTR,24/01/2018,"Companhia Aérea: AZUL, Localizador: V64HTR. Pa...","1553,99",2088475
4,2018,2,ACIR GURGACZ,"Aluguel de imóveis para escritório político, c...",004.948.028-63,GILBERTO PISELO DO NASCIMENTO,002/18,01/02/2018,Despesa com pagamento de aluguel de imóvel par...,6000,2088477
...,...,...,...,...,...,...,...,...,...,...,...
93756,2022,12,ZEQUINHA MARINHO,"Passagens aéreas, aquáticas e terrestres nacio...",22.052.777/0001-32,Exceller Tour,WIXHAI,06/12/2022,"Companhia Aérea: LATAM, Localizador: WIXHAI. P...","2893,04",2191398
93757,2022,12,ZEQUINHA MARINHO,"Passagens aéreas, aquáticas e terrestres nacio...",22.052.777/0001-32,Exceller Tour,WITOLM,09/12/2022,"Companhia Aérea: GOL, Localizador: WITOLM. Pas...","1180,19",2192272
93758,2022,12,ZEQUINHA MARINHO,"Passagens aéreas, aquáticas e terrestres nacio...",22.052.777/0001-32,Exceller Tour,THPKVQ,20/12/2022,"Companhia Aérea: TAM, Localizador: THPKVQ. Pas...","2671,9",2192274
93759,2022,12,ZEQUINHA MARINHO,"Passagens aéreas, aquáticas e terrestres nacio...",22.052.777/0001-32,Exceller Tour,QNN9HX,21/12/2022,"Companhia Aérea: AZUL, Localizador: QNN9HX. Pa...","1334,31",2192244


## Análise Exploratória

In [5]:
# Tipos das colunas

"""Comentários:
    Arrumar tipo de valor reembolsado, object para float
    Arrumar tipo de data, object para date
"""

df_types = df_concat.dtypes.reset_index()
df_types.rename(columns={'index': 'Coluna', 0: 'Tipo'}, inplace=True)
display(title('Tipos das Colunas', 20))
center_df(df_types)

<h1 style="font-size:20px; text-align:center;">Tipos das Colunas</h1>

,Coluna,Tipo
0,ANO,int64
1,MES,int64
2,SENADOR,object
3,TIPO_DESPESA,object
4,CNPJ_CPF,object
5,FORNECEDOR,object
6,DOCUMENTO,object
7,DATA,object
8,DETALHAMENTO,object
9,VALOR_REEMBOLSADO,object


## Criação do Dicionário de Dados


In [6]:
# Dicionário de Dados
titulo = 'Dicionário de Dados'
tamanho_fonte = 24

dataframe = {'Colunas': ['ANO', 
                         'MES', 
                         'SENADOR', 
                         'TIPO_DESPESA', 
                         'CNPJ_CPF', 
                         'FORNECEDOR',
                         'DATA', 
                         'DETALHAMENTO', 
                         'VALOR_REEMBOLSADO'],
            'Descrição': ['Ano da Despesa', 
                          'Mês da Despesa', 
                          'Nome do Senador que realizou a Despesa',
                          'Tipificação da Despesa',
                          'CNPJ ou CPF do executor da Despesa',
                          'Nome do Fornecedor da Despesa',
                          'Data de Apresentação da Despesa',
                          'Detalhamento da Despesa',
                          'Valor a ser reembolsado com a Despesa']}

dic = pd.DataFrame(dataframe)

dic_estilizado = dic.style.set_table_attributes('style="margin-left: auto; margin-right: auto;"')
display(Markdown(f'<h1 style="font-size:{tamanho_fonte}px; text-align:center;">{titulo}</h1>'))
display(HTML(dic_estilizado.render()))


<h1 style="font-size:24px; text-align:center;">Dicionário de Dados</h1>

,Colunas,Descrição
0,ANO,Ano da Despesa
1,MES,Mês da Despesa
2,SENADOR,Nome do Senador que realizou a Despesa
3,TIPO_DESPESA,Tipificação da Despesa
4,CNPJ_CPF,CNPJ ou CPF do executor da Despesa
5,FORNECEDOR,Nome do Fornecedor da Despesa
6,DATA,Data de Apresentação da Despesa
7,DETALHAMENTO,Detalhamento da Despesa
8,VALOR_REEMBOLSADO,Valor a ser reembolsado com a Despesa


In [7]:
df_concat.columns

Index(['ANO', 'MES', 'SENADOR', 'TIPO_DESPESA', 'CNPJ_CPF', 'FORNECEDOR',
       'DOCUMENTO', 'DATA', 'DETALHAMENTO', 'VALOR_REEMBOLSADO',
       'COD_DOCUMENTO'],
      dtype='object')

### Valores Nulos

In [8]:
null_2018 = nulls(data_2018).reset_index()
null_2018.rename(columns={'index': 'Coluna', 0: 'Porcentagem_Nulos'}, inplace=True)

display(title('Porcentagem de Valores Nulos 2018', 24))
center_df(null_2018)

<h1 style="font-size:24px; text-align:center;">Porcentagem de Valores Nulos 2018</h1>

,Coluna,Porcentagem_Nulos
0,ANO,0.000000
1,MES,0.000000
2,SENADOR,0.000000
3,TIPO_DESPESA,0.000000
4,CNPJ_CPF,0.000000
5,FORNECEDOR,0.000000
6,DOCUMENTO,4.148293
7,DATA,0.000000
8,DETALHAMENTO,22.905439
9,VALOR_REEMBOLSADO,0.000000


In [9]:
null_2019 = nulls(data_2019).reset_index()
null_2019.rename(columns={'index': 'Coluna', 0: 'Porcentagem_Nulos'}, inplace=True)

display(title('Porcentagem de Valores Nulos 2019', 24))
center_df(null_2019)

<h1 style="font-size:24px; text-align:center;">Porcentagem de Valores Nulos 2019</h1>

,Coluna,Porcentagem_Nulos
0,ANO,0.000000
1,MES,0.000000
2,SENADOR,0.000000
3,TIPO_DESPESA,0.000000
4,CNPJ_CPF,0.000000
5,FORNECEDOR,0.000000
6,DOCUMENTO,3.392808
7,DATA,0.000000
8,DETALHAMENTO,39.484145
9,VALOR_REEMBOLSADO,0.000000


In [10]:
null_2020 = nulls(data_2020).reset_index()
null_2020.rename(columns={'index': 'Coluna', 0: 'Porcentagem_Nulos'}, inplace=True)

display(title('Porcentagem de Valores Nulos 2020', 24))
center_df(null_2020)

<h1 style="font-size:24px; text-align:center;">Porcentagem de Valores Nulos 2020</h1>

,Coluna,Porcentagem_Nulos
0,ANO,0.000000
1,MES,0.000000
2,SENADOR,0.000000
3,TIPO_DESPESA,0.000000
4,CNPJ_CPF,0.000000
5,FORNECEDOR,0.000000
6,DOCUMENTO,4.400284
7,DATA,0.000000
8,DETALHAMENTO,48.701207
9,VALOR_REEMBOLSADO,0.000000


In [11]:
null_2021 = nulls(data_2021).reset_index()
null_2021.rename(columns={'index': 'Coluna', 0: 'Porcentagem_Nulos'}, inplace=True)

display(title('Porcentagem de Valores Nulos 2021', 24))
center_df(null_2021)

<h1 style="font-size:24px; text-align:center;">Porcentagem de Valores Nulos 2021</h1>

,Coluna,Porcentagem_Nulos
0,ANO,0.000000
1,MES,0.000000
2,SENADOR,0.000000
3,TIPO_DESPESA,0.000000
4,CNPJ_CPF,0.000000
5,FORNECEDOR,0.000000
6,DOCUMENTO,4.011410
7,DATA,0.000000
8,DETALHAMENTO,45.320021
9,VALOR_REEMBOLSADO,0.000000


In [12]:
null_2022 = nulls(data_2022).reset_index()
null_2022.rename(columns={'index': 'Coluna', 0: 'Porcentagem_Nulos'}, inplace=True)

display(title('Porcentagem de Valores Nulos 2022', 24))
center_df(null_2022)

<h1 style="font-size:24px; text-align:center;">Porcentagem de Valores Nulos 2022</h1>

,Coluna,Porcentagem_Nulos
0,ANO,0.000000
1,MES,0.000000
2,SENADOR,0.000000
3,TIPO_DESPESA,0.000000
4,CNPJ_CPF,0.000000
5,FORNECEDOR,0.000000
6,DOCUMENTO,5.580811
7,DATA,0.000000
8,DETALHAMENTO,45.836823
9,VALOR_REEMBOLSADO,0.000000


In [13]:
""" Comentários:
    Atentar para os valores nulos da coluna DETALHAMENTO que tiveram um aumento importante na virada de governo
    Quem são os senadores que mais possuem nulosna coluna DETALHAMENTO?"""
null_concat = nulls(df_concat).reset_index()
null_concat.rename(columns={'index': 'Coluna', 0: 'Porcentagem_Nulos'}, inplace=True)

display(title('Porcentagem de Valores Nulos 2018-2022', 24))
center_df(null_concat)


<h1 style="font-size:24px; text-align:center;">Porcentagem de Valores Nulos 2018-2022</h1>

,Coluna,Porcentagem_Nulos
0,ANO,0.000000
1,MES,0.000000
2,SENADOR,0.000000
3,TIPO_DESPESA,0.000000
4,CNPJ_CPF,0.000000
5,FORNECEDOR,0.000000
6,DOCUMENTO,4.242702
7,DATA,0.000000
8,DETALHAMENTO,38.718657
9,VALOR_REEMBOLSADO,0.000000


### Histograma das Variáveis Categóricas

In [ ]:
#Criação histogramas
plt.figure(figsize=(40, 50))
#Seleção dos valores categóricos
column_list = list(df_concat.columns.values)[2:7]

A = 0

for i in column_list:
    A += 1
    plt.subplot(5, 2, A)
    ax = sns.countplot(data = df_concat.fillna('NaN'), x = i)
    plt.title(i, fontsize = 15)
    for p in ax.patches:
        ax.annotate(f'\n{p.get_height()}', (p.get_x() + 0.4, p.get_height()), ha = 'center', color = 'black', size = 12)
        if A >=6:
            plt.xticks(rotation=45)

plt.tight_layout(h_pad = 2)


### Boxplot Variáveis Numéricas

In [ ]:
# Configurar os subplots
fig, axes = plt.subplots(nrows=1, ncols=5, figsize=(12, 4))  # Define o número de linhas e colunas de subplots e o tamanho da figura

# Utiliza o loop para criar um boxplot para cada ano em um subplot
for i, ano in enumerate(range(2018, 2023)):
    dados_ano = df_concat[df_concat['ANO'] == ano]  # Filtra os dados para o ano atual
    sns.boxplot(data=dados_ano, x='ANO', y='VALOR_REEMBOLSADO', ax=axes[i])  # Cria o boxplot no subplot atual
    axes[i].set_title(f'Ano {ano}', fontsize=12)  # Define o título do subplot
    axes[i].set_xlabel('Ano', fontsize=10)  # Define o rótulo do eixo x do subplot
    axes[i].set_ylabel('Valor Reembolsado', fontsize=10)  # Define o rótulo do eixo y do subplot

plt.tight_layout()  # Ajusta o espaçamento entre os subplots
plt.show()  # Exibe os subplots com os boxplots



## Tratamento dos Dados

In [ ]:
# Retirando as colunas que não serão utilizadas
columns_list = ['DOCUMENTO', 'COD_DOCUMENTO']
df_concat = drop(df_concat, columns_list)
data_2018 = drop(data_2018, columns_list)
data_2019 = drop(data_2019, columns_list)
data_2020 = drop(data_2020, columns_list)
data_2021 = drop(data_2021, columns_list)
data_2022 = drop(data_2022, columns_list)

In [ ]:
# Tratamento dos valores da coluna VALOR_REEMBOLSADO (retirando , e inserindo .) e do tipo da coluna (object to float)
treat_value(df_concat)
treat_value(data_2018)
treat_value(data_2019)
treat_value(data_2020)
treat_value(data_2021)
treat_value(data_2022)

In [ ]:
# Tratamento do tipo da coluna DATA
todate(df_concat)
todate(data_2018)
todate(data_2019)
todate(data_2020)
todate(data_2021)
todate(data_2022)

data_2022.dtypes

### DataFrames de Contagem de Nulos na Coluna Detalhamento por Senador

In [ ]:
# Use o método isnull() para criar uma máscara booleana indicando quais valores são nulos
mask = df_concat['DETALHAMENTO'].isnull()

# Aplique a máscara booleana para filtrar as linhas com valores nulos na coluna 'B'
df_filtrado = df_concat[mask]
df_filtrado['SENADOR'].value_counts()

### DataFrames de Contagem de Solicitações por Senador Por Ano

In [ ]:
# Realizar a contagem de linhas por senador e por ano
contagem = df_concat.groupby(['ANO', 'SENADOR']).size().reset_index(name='CONTAGEM')

In [ ]:
senador_2018 = count(contagem, 2018)
senador_2018

In [ ]:
senador_2019 = count(contagem, 2019)
senador_2019

In [ ]:
senador_2020 = count(contagem, 2020)
senador_2020

In [ ]:
senador_2021 = count(contagem, 2021)
senador_2021

In [ ]:
senador_2022 = count(contagem, 2022)
senador_2022

### DataFrames de Valor Total por Senador e Por Ano

In [ ]:
df = df_concat[['ANO', 'SENADOR', 'VALOR_REEMBOLSADO']]
df

In [ ]:
df = df_concat[['ANO', 'SENADOR', 'VALOR_REEMBOLSADO']]
df = select_year(df, 2018)
value_sum(df)

In [ ]:
df = df_concat[['ANO', 'SENADOR', 'VALOR_REEMBOLSADO']]
df_2019 = select_year(df, 2019)
value_sum(df_2019)

In [ ]:
df = df_concat[['ANO', 'SENADOR', 'VALOR_REEMBOLSADO']]
df_2020 = select_year(df, 2020)
value_sum(df_2020)

### DataFrames de Valor Total por Tipo de Despesa e Por Ano

In [ ]:
df_type_2018 = data_2018[['TIPO_DESPESA', 'VALOR_REEMBOLSADO']]
value_2018 = value_type(df_type_2018)
value_2018

In [ ]:
total_value(value_2018)

In [ ]:
df_type_2019 = data_2019[['TIPO_DESPESA', 'VALOR_REEMBOLSADO']]
value_2019 = value_type(df_type_2019)
value_2019

In [ ]:
total_value(value_2019)

In [ ]:
df_type_2020 = data_2020[['TIPO_DESPESA', 'VALOR_REEMBOLSADO']]
value_2020 = value_type(df_type_2020)
value_2020

In [ ]:
total_value(value_2020)

In [ ]:
df_type_2021 = data_2021[['TIPO_DESPESA', 'VALOR_REEMBOLSADO']]
value_2021 = value_type(df_type_2021)
value_2021

In [ ]:
total_value(value_2021)

In [ ]:
df_type_2022 = data_2022[['TIPO_DESPESA', 'VALOR_REEMBOLSADO']]
value_2022 = value_type(df_type_2022)
value_2022

In [ ]:
total_value(value_2022)